In [1]:
import numpy as np

from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import load
from numpy import asarray
from numpy import expand_dims

import matplotlib.pyplot as plt

from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
from keras.models import load_model
from keras.layers import Input

import cv2

In [2]:
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
MyFaceNet = load_model('facenet_keras.h5')

In [3]:
def create_signature(directory):
    signatures = list()
    for filename in listdir(directory):
        gbr1 = cv2.imread(directory + filename)
    
        wajah = HaarCascade.detectMultiScale(gbr1,1.1,4)
    
        if len(wajah)>0:
            x1, y1, width, height = wajah[0]         
        else:
            x1, y1, width, height = 1, 1, 10, 10

        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height

        gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
        gbr = Image.fromarray(gbr)                  # konversi dari OpenCV ke PIL
        gbr_array = asarray(gbr)

        face = gbr_array[y1:y2, x1:x2]                        

        face = Image.fromarray(face)                       
        face = face.resize((160,160))
        face = asarray(face)

        face = face.astype('float32')
        mean, std = face.mean(), face.std()
        face = (face - mean) / std

        face = expand_dims(face, axis=0)
        signature = MyFaceNet.predict(face)

        signatures.append(signature)
    return signatures

In [4]:
def load_dataset(directory):
    X, y = list(), list()
    
    for subdir in listdir(directory):
        path = directory + subdir + '/'
        
        if not isdir(path):                           # skip any files that might be in the dir
            continue
        
        signatures = create_signature(path)
        labels = [subdir for _ in range(len(signatures))]
        
        X.extend(signatures)
        y.extend(labels)
        
    return asarray(X), asarray(y)

In [5]:
trainX, trainy = load_dataset('training/')

In [7]:
trainX=np.squeeze(trainX, axis=1)

In [11]:
encoder = LabelEncoder()
encoder.fit(trainy)
encoded_Y = encoder.transform(trainy)               # konversi label ke integer

y = np_utils.to_categorical(encoded_Y)         # konversi integer ke one_hot_matrix

In [13]:
model = keras.Sequential()

model.add(layers.Dense(20, input_dim=128, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

In [14]:
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])

In [15]:
model.fit(trainX, y, epochs=100, batch_size=90)

Epoch 1/100
1/1 [==============================] - 0s 2ms/step - loss: 1.2091 - accuracy: 0.3556
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 1.1572 - accuracy: 0.3000
Epoch 3/100
1/1 [==============================] - 0s 6ms/step - loss: 1.1296 - accuracy: 0.2333
Epoch 4/100
1/1 [==============================] - 0s 997us/step - loss: 1.1116 - accuracy: 0.2333
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0958 - accuracy: 0.2667
Epoch 6/100
1/1 [==============================] - 0s 996us/step - loss: 1.0817 - accuracy: 0.3111
Epoch 7/100
1/1 [==============================] - 0s 3ms/step - loss: 1.0700 - accuracy: 0.3778
Epoch 8/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0612 - accuracy: 0.3556
Epoch 9/100
1/1 [==============================] - 0s 4ms/step - loss: 1.0518 - accuracy: 0.3667
Epoch 10/100
1/1 [==============================] - 0s 3ms/step - loss: 1.0432 - accuracy: 0.3889
Epoch 11/100
1/1 [=======

Epoch 84/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2040 - accuracy: 0.9667
Epoch 85/100
1/1 [==============================] - 0s 2ms/step - loss: 0.1980 - accuracy: 0.9667
Epoch 86/100
1/1 [==============================] - 0s 2ms/step - loss: 0.1921 - accuracy: 0.9667
Epoch 87/100
1/1 [==============================] - 0s 2ms/step - loss: 0.1863 - accuracy: 0.9667
Epoch 88/100
1/1 [==============================] - 0s 997us/step - loss: 0.1808 - accuracy: 0.9667
Epoch 89/100
1/1 [==============================] - 0s 2ms/step - loss: 0.1754 - accuracy: 0.9667
Epoch 90/100
1/1 [==============================] - 0s 2ms/step - loss: 0.1702 - accuracy: 0.9667
Epoch 91/100
1/1 [==============================] - 0s 2ms/step - loss: 0.1652 - accuracy: 0.9667
Epoch 92/100
1/1 [==============================] - 0s 6ms/step - loss: 0.1603 - accuracy: 0.9667
Epoch 93/100
1/1 [==============================] - 0s 996us/step - loss: 0.1555 - accuracy: 0.9667
Epoch 94/100
1/1

In [16]:
model.save('modelSmile.h5')